In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as math

In [2]:
#Importerer appendix 3
df_country = pd.read_excel('Appendix_3.xlsx')
df_country

,Country,Language,Licenses,Meaningless_ID
0,United States,American,7865.0,1.0
1,France,French,3410.0,4.0
2,Korea,Korean,2448.0,NaN
3,Italy,Italian,1712.0,7.0
4,Germany,German,1590.0,2.0
...,...,...,...,...
80,Puerto Rico,English,4.0,8.0
81,Liechtenstein,German,3.0,6.0
82,Latvia,Russian,2.0,4.0
83,Kazakhstan,Russian,0.0,3.0


In [3]:
#Fjerner meaningless_ID. Fjerner 'Vesterbro, Københavnsk'. Tjekker om det er fjernet.
del df_country['Meaningless_ID']
df_country = df_country.drop([84])
df_country
#Det er fjernet

,Country,Language,Licenses
0,United States,American,7865.0
1,France,French,3410.0
2,Korea,Korean,2448.0
3,Italy,Italian,1712.0
4,Germany,German,1590.0
...,...,...,...
79,Nepal,Nepali,6.0
80,Puerto Rico,English,4.0
81,Liechtenstein,German,3.0
82,Latvia,Russian,2.0


In [4]:
#Sorterer data efter sprog i alfabetisk rækkefølge for at skabe overblik
df_country.head(50).sort_values(['Language'], ascending=True)

,Country,Language,Licenses
0,United States,American,7865.0
46,Morocco,Arabic,44.0
48,Egypt,Arabic,40.0
26,Lebanon,Arabic,137.0
6,China,Chineese,1293.0
44,Croatia,Croatian,49.0
36,Czech Republic,Czech,78.0
37,South Africa,English,74.0
18,Norway,English,274.0
31,New Zealand,English,108.0


In [5]:
#Ensarter og korrigerer værdier i sprog-kollonnen, da det varierer, hvordan sprog er formuleret eller angivet
df_country['Language'] = df_country['Language']\
    .replace(['American'],'English')\
    .replace(['Mandarin(Chinese)'], 'Chinese')\
    .replace(['spanish'], 'Spanish')\
    .replace(['chineese'], 'Chinese')\
    .replace(['Chineese'], 'Chinese')

df_country_sort = df_country.sort_values(['Language'], ascending=True)

In [6]:
#Sikrer at 'Country er en string-value'
df_country_sort['Country']=df_country_sort['Country'].astype("string")
df_country_sort.dtypes

Country      string
Language     object
Licenses    float64
dtype: object

In [7]:
#Grupperer lande efter deres geografiske lokalisation for senere at allokere supportcentre, der er svarende til tidszoner
def country_cat(variable):
    if variable in ['Lithuania','Monaco','Croatia','France','Italy','Germany','United Kingdom','Spain','Netherlands','Ireland','Poland','Denmark','Switzerland','Sweden','Norway','Belgium','Austria','Slovakia','Greece','Romania','Czech Republic','Finland','Lithaunia','Hungary','Ukraine','Crotia','Iceland','Bulgaria','Luxembourg','Serbia','Slovenia','Estonia','Portugal','Cyprus','Armenien','Liechtenstein','Latvia']:
        return 'Europe'
    if variable in ['Israel','United Arab Emirates','Lebanon','Pakistan','Kuwait','Lebanon','Morocco','Egypt','Tunisia','Saudi Arabia','Jordan','Algeria','Iraq','Turkey','Iran']:
        return 'Middle East'
    if variable in ['Thailand','Singapore','Indonesia','Australia','New Zealand','China','Korea','Philippines','Myanmar','Malaysia','India','Japan','Taiwan','Hong Kong SAR','Nepal','Vietnam']:
        return 'Pacific'
    if variable in ['Armenia','Moldova','Georgia','Azerbaijan','Kazakhstan','Russia']:
        return 'Russian'
    if variable in ['United States','Canada']:
        return 'North America'
    if variable in ['South Africa']:
        return 'Africa'
    else:
        return 'South America'
    
    
#Fjerner de mellemrum, der er i starten af landenes navne, så definitionen ovenfor kan virke   
df_country_sort['Country']= df_country_sort['Country'].str.lstrip()

#Indsætter område som ny kollonne i dataframe
df_country_sort['Area'] = df_country_sort['Country'].apply(country_cat)
df_country_sort.tail(60)

,Country,Language,Licenses,Area
0,United States,English,7865.0,North America
21,Belgium,English,234.0,Europe
80,Puerto Rico,English,4.0,South America
5,Australia,English,1493.0,Pacific
7,United Kingdom,English,1221.0,Europe
9,Canada,English,782.0,North America
10,Netherlands,English,659.0,Europe
76,Malaysia,English,6.0,Pacific
13,Denmark,English,380.0,Europe
11,Ireland,English,634.0,Europe


In [8]:
#Her parres sprog med geografisk lokalisation, så det er nemmere at finde ud af, hvilket supportcenter, der kan knyttes til hvert land
df_country_sort["Language & area"] = df_country_sort[["Language", "Area"]].agg(', '.join, axis=1)
df_country_sort.head(60)

,Country,Language,Licenses,Area,Language & area
48,Egypt,Arabic,40.0,Middle East,"Arabic, Middle East"
73,Algeria,Arabic,9.0,Middle East,"Arabic, Middle East"
26,Lebanon,Arabic,137.0,Middle East,"Arabic, Middle East"
65,Jordan,Arabic,9.0,Middle East,"Arabic, Middle East"
59,Saudi Arabia,Arabic,15.0,Middle East,"Arabic, Middle East"
56,Tunisia,Arabic,20.0,Middle East,"Arabic, Middle East"
78,Iraq,Arabic,6.0,Middle East,"Arabic, Middle East"
46,Morocco,Arabic,44.0,Middle East,"Arabic, Middle East"
50,Bulgaria,Bulgarian,29.0,Europe,"Bulgarian, Europe"
6,China,Chinese,1293.0,Pacific,"Chinese, Pacific"


In [9]:
#Her er en liste over unikke værdier i ovenstående dataframe, hvad angår land/område-kombinationer.
#Dette bruges til at skabe overblik over, hvordan supportcentre kan allokeres
df_country_sort['Language & area'].unique()

array(['Arabic, Middle East', 'Bulgarian, Europe', 'Chinese, Pacific',
       'Croatian, Europe', 'Czech, Europe', 'English, Europe',
       'English, Africa', 'English, Pacific', 'English, Middle East',
       'English, North America', 'English, South America',
       'French, Europe', 'Georgian, Russian', 'German, Europe',
       'Greek, Europe', 'Hungarian, Europe', 'Indonesian, Pacific',
       'Italian, Europe', 'Japanese, Pacific', 'Korean, Pacific',
       'Lithanian, Europe', 'Nepali, Pacific', 'Persian, Middle East',
       'Polish, Europe', 'Portuguese, South America',
       'Portuguese, Europe', 'Romanian, Europe', 'Russian, Europe',
       'Russian, Russian', 'Serbian, Europe', 'Slovak, Europe',
       'Slovenian, Europe', 'Spanish, Europe', 'Spanish, South America',
       'Thai, Pacific', 'Turkish, Middle East', 'Vietnamese, Pacific'],
      dtype=object)

In [10]:
#Nu fordeler vi de enkelte sprogområder til et specifikt supportcenter
#Danmark udelades, da det 1) er dyrt, 2) taler relativt få sprog og 3) Ukraine kan dække nogenlunde samme områder (og Ukraine har flere sprog)
def country_service(varcomb):
    if varcomb in ['Chinese, Pacific','English, Pacific','Japanese, Pacific', 'Korean, Pacific']:
        return 'China'
    if varcomb in ['English, Europe', 'English, Africa','English, Middle East','French, Europe','German, Europe','Italian, Europe','Russian, Europe','Russian, Russian','Spanish, Europe']:
        return 'Ukraine'
    if varcomb in ['English, North America', 'English, South America']:
        return 'USA'
    if varcomb in ['Portuguese, South America','Spanish, South America']:
        return 'Colombia'
    else:
        return np.nan
    
#Vi forudsætter hermed, at man i USA's supportcenter ikke kan tale spansk eller portugesisk
        
#Indsætter ny kollone, hvor hvert land har fået tildelt et supportcenter. NaN betyder, at der ikke kan tilknytttes et supportcenter.
#Der er taget hensyn til tidszoner
df_country_sort['Support Center'] = df_country_sort['Language & area'].apply(country_service)
df_country_sort.head(60)

,Country,Language,Licenses,Area,Language & area,Support Center
48,Egypt,Arabic,40.0,Middle East,"Arabic, Middle East",NaN
73,Algeria,Arabic,9.0,Middle East,"Arabic, Middle East",NaN
26,Lebanon,Arabic,137.0,Middle East,"Arabic, Middle East",NaN
65,Jordan,Arabic,9.0,Middle East,"Arabic, Middle East",NaN
59,Saudi Arabia,Arabic,15.0,Middle East,"Arabic, Middle East",NaN
56,Tunisia,Arabic,20.0,Middle East,"Arabic, Middle East",NaN
78,Iraq,Arabic,6.0,Middle East,"Arabic, Middle East",NaN
46,Morocco,Arabic,44.0,Middle East,"Arabic, Middle East",NaN
50,Bulgaria,Bulgarian,29.0,Europe,"Bulgarian, Europe",NaN
6,China,Chinese,1293.0,Pacific,"Chinese, Pacific",China


In [11]:
#Fjerner NaN-værdier (dvs. lande der ikke sprogligt kan dækkes)
df_support = df_country_sort.dropna(axis=0)
df_support.head(60)

,Country,Language,Licenses,Area,Language & area,Support Center
6,China,Chinese,1293.0,Pacific,"Chinese, Pacific",China
28,Hong Kong SAR,Chinese,112.0,Pacific,"Chinese, Pacific",China
20,Taiwan,Chinese,263.0,Pacific,"Chinese, Pacific",China
38,Finland,English,74.0,Europe,"English, Europe",Ukraine
37,South Africa,English,74.0,Africa,"English, Africa",Ukraine
24,India,English,191.0,Pacific,"English, Pacific",China
49,Kuwait,English,35.0,Middle East,"English, Middle East",Ukraine
43,Pakistan,English,54.0,Middle East,"English, Middle East",Ukraine
31,New Zealand,English,108.0,Pacific,"English, Pacific",China
67,Myanmar,English,9.0,Pacific,"English, Pacific",China


In [12]:
#Udregner hvor mange supportcentre, der kan ligge pr. land
#Her antager vi, at etableringsomkostningerne i de enkelte lande = 0 
df_support.groupby('Support Center')['Licenses'].sum()

Support Center
China        6392.0
Colombia      490.0
USA          8651.0
Ukraine     12632.0
Name: Licenses, dtype: float64

In [13]:
#Indlæser data fra appendix 2
#Antagelse: Totale omkostninger er indfanget (imputed) i medarbejderomkostninger
App2 = pd.read_excel('Appendix_2.xlsx')
a2 = App2.drop([5])
a2['Avg Annual Employee cost'] = (a2['Total cost']/a2['Average FTE'])
a2['Avg Monthly Employee cost'] = a2['Avg Annual Employee cost']/12
a2['Support Center'] = a2['Support Center'].astype("string")
a2

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost
0,Denmark,2150000,5,430000.0,35833.333333
1,China,2500000,10,250000.0,20833.333333
2,USA,750000,9,83333.333333,6944.444444
3,Colombia,2100000,21,100000.0,8333.333333
4,Ukraine,650000,38,17105.263158,1425.438596


In [14]:
#Laver variable for de 30 % som ikke længere går til reseller, men som går til 3shape direkte
profit_margin = 2000*0.3

In [15]:
#Udregner profit-loss balance for hvert land pba. tallen fra ovenstående dataframe.
#Vi udregner ud fra, hvad det minimale antal medarbejdere, der skal til for at dække licenserne. Dette giver det mest omkostningseffektive resultat.
a4 = pd.DataFrame({'No. of employees': np.arange(3,43,1), 'Licenses serviced': np.arange(300, 8200, 200)})
a4['Extra revenue'] = a4['Licenses serviced']*profit_margin
a4['PL-balance_DK'] = a4['Extra revenue'] - (430000*a4['No. of employees'])
a4['PL-balance_CN'] = a4['Extra revenue'] - (250000*a4['No. of employees'])
a4['PL-balance_US'] = a4['Extra revenue'] - (83333.3*a4['No. of employees'])
a4['PL-balance_CO'] = a4['Extra revenue'] - (100000*a4['No. of employees'])
a4['PL-balance_UKR'] = a4['Extra revenue'] - (17105.3*a4['No. of employees'])
a4.astype(int).round(0)

,No. of employees,Licenses serviced,Extra revenue,PL-balance_DK,PL-balance_CN,PL-balance_US,PL-balance_CO,PL-balance_UKR
0,3,300,180000,-1110000,-570000,-69999,-120000,128684
1,4,500,300000,-1420000,-700000,-33333,-100000,231578
2,5,700,420000,-1730000,-830000,3333,-80000,334473
3,6,900,540000,-2040000,-960000,40000,-60000,437368
4,7,1100,660000,-2350000,-1090000,76666,-40000,540262
5,8,1300,780000,-2660000,-1220000,113333,-20000,643157
6,9,1500,900000,-2970000,-1350000,150000,0,746052
7,10,1700,1020000,-3280000,-1480000,186667,20000,848947
8,11,1900,1140000,-3590000,-1610000,223333,40000,951841
9,12,2100,1260000,-3900000,-1740000,260000,60000,1054736


In [16]:
#Indsætter antallet af licenser som hvert datacenter potentielt kan dække. Tallene er trukket fra den tidligere groupby-funktion
a2['Number of potential licenses'] = [0, 6392,8651,490,12638]
a2.astype(dtype = int, errors = 'ignore').round(0)

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses
0,Denmark,2150000,5,430000,35833,0
1,China,2500000,10,250000,20833,6392
2,USA,750000,9,83333,6944,8651
3,Colombia,2100000,21,100000,8333,490
4,Ukraine,650000,38,17105,1425,12638


In [17]:
#Indsætter det minimale antal af ansatte, der behøves for at dække ALLE licenser
a2['Number of employees needed'] = [0,34,45,4,64]
a2.astype(dtype = int, errors = 'ignore').round(0)

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses,Number of employees needed
0,Denmark,2150000,5,430000,35833,0,0
1,China,2500000,10,250000,20833,6392,34
2,USA,750000,9,83333,6944,8651,45
3,Colombia,2100000,21,100000,8333,490,4
4,Ukraine,650000,38,17105,1425,12638,64


In [18]:
#Udregner hvad totale medarbejderomkostningerne er pr. år såfremt ALLE licenser bliver dækket
a2['Total employee cost'] = a2['Number of employees needed']*a2['Avg Annual Employee cost']
a2.astype(dtype = int, errors = 'ignore').round(0)

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses,Number of employees needed,Total employee cost
0,Denmark,2150000,5,430000,35833,0,0,0
1,China,2500000,10,250000,20833,6392,34,8500000
2,USA,750000,9,83333,6944,8651,45,3750000
3,Colombia,2100000,21,100000,8333,490,4,400000
4,Ukraine,650000,38,17105,1425,12638,64,1094736


In [19]:
#Da Danmark ikke får tildelt et support center (på kort sigt) grundet de høje omkostninger, fjernes Danmark fra dataframe
a2_drop_Denmark = a2.drop([0], axis=0).astype(dtype = int, errors = 'ignore').round(0)
a2_drop_Denmark

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses,Number of employees needed,Total employee cost
1,China,2500000,10,250000,20833,6392,34,8500000
2,USA,750000,9,83333,6944,8651,45,3750000
3,Colombia,2100000,21,100000,8333,490,4,400000
4,Ukraine,650000,38,17105,1425,12638,64,1094736


In [20]:
#Udregner differensen mellem ekstra indtjening ved insourcing og de totale medarbejderomkostninger
a2_drop_Denmark['Profit/loss']=(a2_drop_Denmark['Number of potential licenses']*profit_margin)-a2_drop_Denmark['Total employee cost']
#Udregner omkostning pr. licens (unit cost) fordelt på lande
a2_drop_Denmark['Unit cost']=a2_drop_Denmark['Total employee cost']/a2_drop_Denmark['Number of potential licenses']

a2_drop_Denmark.astype(dtype = int, errors = 'ignore').round(0)

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses,Number of employees needed,Total employee cost,Profit/loss,Unit cost
1,China,2500000,10,250000,20833,6392,34,8500000,-4664800,1329
2,USA,750000,9,83333,6944,8651,45,3750000,1440600,433
3,Colombia,2100000,21,100000,8333,490,4,400000,-106000,816
4,Ukraine,650000,38,17105,1425,12638,64,1094736,6488064,86


In [21]:
#Udregner netto-profitten/tabet såfremt alle supportcentre etableres med support på ALLE licenser
a2_drop_Denmark['Profit/loss'].sum()

3157864.0

In [22]:
#Der er netto-profit på ca. 3.158.000, såfremt alle supportcentre etableres

In [23]:
                                           
    
                                            ####UDREGNING AF ADOPTION RATES####
    
    

In [24]:
#Nu beregnes omkostningerne på forskellige adoption-rates over tre år: HHv. 10%, 50% og 100% af licenserne dækkes af customer first
#Antagelse: Væksten er 10 procent pr. år. (dvs. antallet af licenser stiger med 10 procent pr. år)

#10% adoption
a2_drop_Denmark['10% adoption, y1']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*0.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['10% adoption, y2']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*1.1*0.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['10% adoption, y3']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*1.1*1.1*0.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']

#50% adoption
a2_drop_Denmark['50% adoption, y1']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*0.5)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['50% adoption, y2']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*0.5*1.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['50% adoption, y3']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*1.1*1.1*0.5)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']

#100% adoption
a2_drop_Denmark['100% adoption, y1']=np.ceil((((a2_drop_Denmark['Number of potential licenses'])-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['100% adoption, y2']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*1.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']
a2_drop_Denmark['100% adoption, y3']=np.ceil((((a2_drop_Denmark['Number of potential licenses']*1.1*1.1)-300)/200)+3)*a2_drop_Denmark['Avg Annual Employee cost']

a2_drop_Denmark.astype(dtype = int, errors = 'ignore').round(0)

,Support Center,Total cost,Average FTE,Avg Annual Employee cost,Avg Monthly Employee cost,Number of potential licenses,Number of employees needed,Total employee cost,Profit/loss,Unit cost,"10% adoption, y1","10% adoption, y2","10% adoption, y3","50% adoption, y1","50% adoption, y2","50% adoption, y3","100% adoption, y1","100% adoption, y2","100% adoption, y3"
1,China,2500000,10,250000,20833,6392,34,8500000,-4664800,1329,1250000,1500000,1500000,4500000,5000000,5250000,8500000,9250000,10250000
2,USA,750000,9,83333,6944,8651,45,3750000,1440600,433,499998,583331,583331,1999992,2166658,2333324,3749985,4166650,4499982
3,Colombia,2100000,21,100000,8333,490,4,400000,-106000,816,200000,200000,200000,300000,300000,300000,400000,500000,500000
4,Ukraine,650000,38,17105,1425,12638,64,1094736,6488064,86,136840,153945,171050,581570,632885,684200,1111825,1231560,1334190
